# Load packages


In [228]:
# 'private_twitter_credentials' contains my Twitter credentials. Replace by 'twitter_credentials' with your credentials
import private_twitter_credentials
import twitter
import datetime
import pandas as pd
import time

TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

# Seeting up twitter authentication

In [229]:
consumer_key = private_twitter_credentials.consumer_key
consumer_secret = private_twitter_credentials.consumer_secret
access_token = private_twitter_credentials.access_token
access_token_secret = private_twitter_credentials.access_token_secret

api = twitter.Api(
    consumer_key         =   consumer_key,
    consumer_secret      =   consumer_secret,
    access_token_key     =   access_token,
    access_token_secret  =   access_token_secret,
    tweet_mode = 'extended'
)

# Functions

Class `TweetMiner` class contains two methods: 

* `mine_user_tweets` which mine user's tweets making use of [Get user_timeline](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline)

* `search_tweets` which mine tweets using [GetSearch](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)

`search_tweets` gives you possibility to perform queries. You can for instances perform a search at Twitter and copy what comes after `q` in your browser.

For example, if I search `@picnic @JumboSupermarkt @albertheijn covid-19` the query used as argument in `search_tweets` is `q=%40picnic%20%40JumboSupermarkt%20%40albertheijn%20covid-19&src=typed_query`. 


In [230]:
import datetime

class TweetMiner(object):
    """ Make possible obtaining tweets using twitter user id (mine_user_tweets) or performing a standard Twitter 
    API search"""

    
    def __init__(self, api, result_limit = 20, max_pages = 40):
        """result_limit = count that can take max 200 (mine_user_tweets) and max 100 (search_tweets)"""
        
        self.api = api        
        self.result_limit = result_limit
        self.max_pages = max_pages
        

    def mine_user_tweets(self, user, mine_retweets=False):
        """ Mine tweets of user = screen_name or user_id"""

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= self.max_pages:
            
            if last_tweet_id:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, max_id=last_tweet_id - 1, 
                                                        include_rts=mine_retweets)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses   =   self.api.GetUserTimeline(screen_name=user, count=self.result_limit, 
                                                        include_rts=mine_retweets)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'mined_at':         datetime.datetime.now(),
                        'created_at':       item['created_at'],
                        'tweet_id':         item['id'],
                        'tweet_id_str':     item['id_str'],
                        'screen_name':      item['user']['screen_name'],
                        'favorite_count':   item['favorite_count'],
                        'text':             item['full_text'],
                        'source':           item['source'],
                        'language':         item['lang'],
                        'retweet_count':    item['retweet_count'],
                        #user info
                        'user_favourites_count': item['user']['favourites_count'],
                        'followers_count':  item['user']['followers_count'],
                        'friends_count':    item['user']['friends_count']
                    }
            
                
                except:
                    mined = {
                        'mined_at':         datetime.datetime.now(),
                        'created_at':       item['created_at'],
                        'tweet_id':         item['id'],
                        'tweet_id_str':     item['id_str'],
                        'screen_name':      item['user']['screen_name'],
#                         'favorite_count':   item['favorite_count'],
                        'text':             item['full_text'],
                        'source':           item['source'],
                        'language':         item['lang'],
                        'retweet_count':    0,
                        # user info
                        'user_favourites_count': item['user']['favourites_count'],
                        'followers_count':  item['user']['followers_count'],
                        'friends_count':    item['user']['friends_count']
                        }
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data
    
    def search_tweets(max_pages = 20, count = 20, raw_query = None, result_type = 'mixed'):
        """ Search tweets """

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            
            if last_tweet_id:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type, 
                                         max_id=last_tweet_id - 1)
                statuses = [ _.AsDict() for _ in statuses]
            else:
                statuses = api.GetSearch(raw_query=raw_query, count = count, result_type=result_type)
                statuses = [_.AsDict() for _ in statuses]
                
            for item in statuses:
                # Using try except here.
                # When retweets = 0 we get an error (GetUserTimeline fails to create a key, 'retweet_count')
                try:
                    mined = {
                        'mined_at':                datetime.datetime.now(),
                        'created_at':              item['created_at'],
                        'tweet_id':                item['id'],
                        'tweet_id_str':            item['id_str'],
                        'in_reply_to_screen_name': item['in_reply_to_screen_name'],
                        'in_reply_to_status_id':   item['in_reply_to_status_id'],
                        'in_reply_to_user_id':     item['in_reply_to_user_id'],
                        'language':                item['lang'],
                        'text':                    item['full_text'],
                        'hashtags':                item['hashtags'],
                        'source':                  item['source'],
                       # info about user
                        'screen_name':             item['user']['screen_name'],
                        'user_tweet_id':           item['user']['id'],
                        'user_tweet_id_str':       item['user']['id_str'],
                        'user_favourites_count':   item['user']['favourites_count'],
                        'followers_count':         item['user']['followers_count'],
                        'friends_count':           item['user']['friends_count']
                    }
                    
                except:
                    mined = {
                        'mined_at':                datetime.datetime.now(),
                        'created_at':              item['created_at'],
                        'tweet_id':                item['id'],
                        'tweet_id_str':            item['id_str'],
#                         'in_reply_to_screen_name': item['in_reply_to_screen_name'],
#                         'in_reply_to_status_id':   item['in_reply_to_status_id'],
#                         'in_reply_to_user_id':     item['in_reply_to_user_id'],
                        'language':                item['lang'],
                        'text':                    item['full_text'],
                        'hashtags':                item['hashtags'],
                        'source':                  item['source'],
                       # info about user
                        'screen_name':             item['user']['screen_name'],
                        'user_tweet_id':           item['user']['id'],
                        'user_tweet_id_str':       item['user']['id_str'],
                        'user_favourites_count':   item['user']['favourites_count'],
                        'followers_count':         item['user']['followers_count'],
                        'friends_count':           item['user']['friends_count']
                    }
                                            
                
                last_tweet_id = item['id']
                data.append(mined)
                
            page += 1
            
        return data

In [261]:
def process_and_save(df, file_name, mine_user_twitter=1):
    """ Save retrieved tweets in csv file.
    
    Input:
    
    df : dataframe of tweets'data
    file_name: name with which the csv will be saved (without extension)
    mine_user_twitter: Indicates if df came contains tweets from a twitter user, i.e., was obtained using 
    GetUserTimeline since the information obtained from this method is different from an API search from GetSearch
    
    """
    
    TodaysDate = time.strftime("%Y-%m-%d-%H-%M")

    
    # Create columns 'year', 'month', 'day', 'hour', 'min' from 'created_at'
    df['created_at'] = pd.to_datetime(df['created_at'])
        
    df['year'] = df['created_at'].dt.year 
    df['month'] = df['created_at'].dt.month 
    df['day'] = df['created_at'].dt.day 
    df['hour'] = df['created_at'].dt.hour 
    df['minute'] = df['created_at'].dt.minute
    df['day_of_week'] = df['created_at'].dt.weekday
    
    if mine_user_twitter:
    
        df = df[['mined_at', 'created_at', 'year', 'month', 'day','day_of_week', 
             'hour', 'minute', 'screen_name', 'tweet_id', 'tweet_id_str',  'retweet_count', 'favorite_count', 'source', 'language', 'user_favourites_count', 
             'followers_count','friends_count','text']]
    else:
        df = df[['mined_at','created_at', 'year', 'month', 'day','day_of_week', 
                 'hour', 'minute', 'tweet_id', 'tweet_id_str', 'in_reply_to_screen_name','in_reply_to_status_id',
                 'in_reply_to_user_id', 'hashtags','source','language', 'screen_name','user_tweet_id','user_tweet_id_str','user_favourites_count','followers_count',
                 'friends_count', 'text']]
        
    df.sort_values(by='created_at',inplace = True)
    df = df.loc[df.astype(str).drop_duplicates(subset=['created_at','tweet_id','text']).index]
    df.reset_index(drop = True, inplace = True)
    
    
    df.to_csv("../data/tweets/"+file_name+"_"+TodaysDate+".csv", index = False)
    
    return df
    
    

# Retrieving twitter data

The goal of this project is to check the sentiment of users towards the main providers of online grocery shopping, i.e., Jumbo Supermarkten, AH, and Picnic.

Everything changed since the first case of corona virus in The Netherlands (February 27th) and the way of shopping groceries suffered important change with a jump in number of users that opted for online grocery shopping. Supermarkts were not ready for such a explosion of demand, some adapted faster than others. Specially, `Picnic` that has the sole focus on online shopping.

I have my own experiences but I want to via tweets messages over these 3 providers of online grocery shopping get the sentiment of the users in this 'special' moments faced by both consumers and providers.

The idea is to get twitters covering the period from the 1st case until today both for info retrieved by user (`GetUserTimeline`) and by query (`GetSearch`).

Although, it is not possible to have control over the period covered by the search, we will play with parameters and go as far as possible.

## Getting twitter by user

To start we will obtain tweets for `picnic`, `JumboSupermarkt`, and `albertheijn` which are the tweet screen name of the 3 providers of online grocery shopping.


### Picnic

In [232]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 100)
picnic = miner.mine_user_tweets(user="picnic")
df_picnic = process_and_save(pd.DataFrame(picnic), "picnic")

In [233]:
df_picnic.tail()

,mined_at,created_at,year,month,day,day_of_week,hour,minute,screen_name,tweet_id,tweet_id_str,retweet_count,favorite_count,source,language,user_favourites_count,followers_count,friends_count,text
1990,2020-06-21 13:16:24.115881,2020-06-18 17:04:24+00:00,2020,6,18,3,17,4,picnic,1273662862096060417,1273662862096060417,0,NaN,"<a href=""http://www.zendesk.com"" rel=""nofollow...",nl,3881,4843,5,"@rolandweyers Ah, dat is inderdaad een dode mu..."
1991,2020-06-21 13:16:24.115881,2020-06-19 09:59:09+00:00,2020,6,19,4,9,59,picnic,1273918233834393602,1273918233834393602,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",nl,3881,4843,5,@MiriamVermeulen Wat fijn dat je al 100 plekke...
1992,2020-06-21 13:16:24.115881,2020-06-19 10:09:21+00:00,2020,6,19,4,10,9,picnic,1273920800064757760,1273920800064757760,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",nl,3881,4843,5,@MrsVlamingo De Presto maaltijden variëren af ...
1993,2020-06-21 13:16:24.115881,2020-06-20 09:40:19+00:00,2020,6,20,5,9,40,picnic,1274275880756482049,1274275880756482049,0,NaN,"<a href=""http://www.zendesk.com"" rel=""nofollow...",nl,3881,4843,5,@Mr_Widewood Goedemorgen! Goed dat je dit even...
1994,2020-06-21 13:16:24.115881,2020-06-20 19:56:51+00:00,2020,6,20,5,19,56,picnic,1274431035896467466,1274431035896467466,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",es,3881,4843,5,@dbenshachar ^Lino https://t.co/XK06YZyXVh


In [234]:
df_picnic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1995 entries, 0 to 1994
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               1995 non-null   datetime64[ns]     
 1   created_at             1995 non-null   datetime64[ns, UTC]
 2   year                   1995 non-null   int64              
 3   month                  1995 non-null   int64              
 4   day                    1995 non-null   int64              
 5   day_of_week            1995 non-null   int64              
 6   hour                   1995 non-null   int64              
 7   minute                 1995 non-null   int64              
 8   screen_name            1995 non-null   object             
 9   tweet_id               1995 non-null   int64              
 10  tweet_id_str           1995 non-null   object             
 11  retweet_count          1995 non-null   int64            

In [235]:
min(df_picnic.created_at),max(df_picnic.created_at)

(Timestamp('2019-07-13 18:42:42+0000', tz='UTC'),
 Timestamp('2020-06-20 19:56:51+0000', tz='UTC'))

For `Picnic` we went far enough. We got data back to July 2019!

In [236]:
print("Picnic's followers", df_picnic.loc[df_picnic.shape[0]-1,'followers_count'])
print("Picnic's friends", df_picnic.loc[df_picnic.shape[0]-1,'friends_count'])

Picnic's followers 4843
Picnic's friends 5


In [237]:
df_picnic['language'].value_counts()

nl     1780
und     145
en       53
es        5
de        3
in        2
fr        2
it        1
ht        1
cy        1
sv        1
fi        1
Name: language, dtype: int64

### JumboSupermarkt

In [238]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=20, max_pages = 200)
JumboSupermarkt = miner.mine_user_tweets(user="JumboSupermarkt")
df_JumboSupermarkt = processing_and_saving(pd.DataFrame(JumboSupermarkt), "JumboSupermarkt")


In [239]:
df_JumboSupermarkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3230 entries, 0 to 3229
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3230 non-null   datetime64[ns]     
 1   created_at             3230 non-null   datetime64[ns, UTC]
 2   year                   3230 non-null   int64              
 3   month                  3230 non-null   int64              
 4   day                    3230 non-null   int64              
 5   day_of_week            3230 non-null   int64              
 6   hour                   3230 non-null   int64              
 7   minute                 3230 non-null   int64              
 8   screen_name            3230 non-null   object             
 9   tweet_id               3230 non-null   int64              
 10  tweet_id_str           3230 non-null   object             
 11  retweet_count          3230 non-null   int64            

In [240]:
min(df_JumboSupermarkt.created_at),max(df_JumboSupermarkt.created_at)

(Timestamp('2020-03-06 19:12:05+0000', tz='UTC'),
 Timestamp('2020-06-21 09:33:32+0000', tz='UTC'))

In [241]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=25, max_pages = 250)
JumboSupermarkt = miner.mine_user_tweets(user="JumboSupermarkt")
df_JumboSupermarkt = process_and_save(pd.DataFrame(JumboSupermarkt), "JumboSupermarkt")


In [242]:
min(df_JumboSupermarkt.created_at),max(df_JumboSupermarkt.created_at)

(Timestamp('2020-03-07 20:15:43+0000', tz='UTC'),
 Timestamp('2020-06-21 09:33:32+0000', tz='UTC'))

Changing parameters does not seem to help here and we could only retrieve data from March 6th, 2020 until now for Jumbo.

In [243]:
print("Jumbo's followers", df_JumboSupermarkt.loc[df_JumboSupermarkt.shape[0]-1,'followers_count'])
print("Jumbo's friends", df_JumboSupermarkt.loc[df_JumboSupermarkt.shape[0]-1,'friends_count'])

Jumbo's followers 16212
Jumbo's friends 1710


In [244]:
df_JumboSupermarkt['language'].value_counts()

nl     3112
en       68
und      25
in        3
de        2
da        1
tl        1
Name: language, dtype: int64

### albertheijn

In [252]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=200, max_pages = 20)
albertheijn = miner.mine_user_tweets(user="albertheijn")
df_albertheijn = process_and_save(pd.DataFrame(albertheijn), "albertheijn")


In [253]:
df_albertheijn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3218 entries, 0 to 3217
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3218 non-null   datetime64[ns]     
 1   created_at             3218 non-null   datetime64[ns, UTC]
 2   year                   3218 non-null   int64              
 3   month                  3218 non-null   int64              
 4   day                    3218 non-null   int64              
 5   day_of_week            3218 non-null   int64              
 6   hour                   3218 non-null   int64              
 7   minute                 3218 non-null   int64              
 8   screen_name            3218 non-null   object             
 9   tweet_id               3218 non-null   int64              
 10  tweet_id_str           3218 non-null   object             
 11  retweet_count          3218 non-null   int64            

In [254]:
min(df_albertheijn.created_at),max(df_albertheijn.created_at)

(Timestamp('2020-04-03 15:35:47+0000', tz='UTC'),
 Timestamp('2020-06-21 12:27:32+0000', tz='UTC'))

In [255]:
# Result limit == count parameter from our GetUserTimeline() it can take max 200
# More pages more back in time you can go
miner = TweetMiner(api, result_limit=200, max_pages = 100)
albertheijn = miner.mine_user_tweets(user="albertheijn")
df_albertheijn = process_and_save(pd.DataFrame(albertheijn), "albertheijn")


In [256]:
min(df_albertheijn.created_at),max(df_albertheijn.created_at)

(Timestamp('2020-04-03 15:35:47+0000', tz='UTC'),
 Timestamp('2020-06-21 12:45:16+0000', tz='UTC'))

In [257]:
df_albertheijn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3219 entries, 0 to 3218
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype              
---  ------                 --------------  -----              
 0   mined_at               3219 non-null   datetime64[ns]     
 1   created_at             3219 non-null   datetime64[ns, UTC]
 2   year                   3219 non-null   int64              
 3   month                  3219 non-null   int64              
 4   day                    3219 non-null   int64              
 5   day_of_week            3219 non-null   int64              
 6   hour                   3219 non-null   int64              
 7   minute                 3219 non-null   int64              
 8   screen_name            3219 non-null   object             
 9   tweet_id               3219 non-null   int64              
 10  tweet_id_str           3219 non-null   object             
 11  retweet_count          3219 non-null   int64            

Again the changing parameters didn't work. For `AH` we only succeeded in getting back to April 3th, 2020.

In [259]:
print("AH's followers", df_albertheijn.loc[df_albertheijn.shape[0]-1,'followers_count'])
print("AH's friends", df_albertheijn.loc[df_albertheijn.shape[0]-1,'friends_count'])

AH's followers 45523
AH's friends 5


In [260]:
df_albertheijn['language'].value_counts()

nl     3111
en       74
und       7
et        5
fr        5
tr        4
fi        3
in        2
da        2
pl        1
de        1
is        1
ht        1
sv        1
es        1
Name: language, dtype: int64

# Applying GetSearch to search for a defined query


Twitter’s search parameters are a bit complex, to perform a particular search, you can consult Twitter’s documentation at https://dev.twitter.com/rest/public/search.

An easier way is to make use of [Twitter’s Advanced Search](toolhttps://twitter.com/search-advanced), and then use the part of search URL after the "?" to use `raw_query`, removing the `&src=typd` portion.

In this section we will perform some queries to allow us to compare `Picnic`, `Jumbo`, and `AH`. In particular, my main goal is to include also `COVID-19`. But since we are in June and things seem to be now much more in control than few months ago I'm not sure if I'll be able to retrieve enough Tweets using these key words.

In addition, the results obtained from Twitter API searches against a sampling of recent Tweets published in the past 7 days (more details [here](https://developer.twitter.com/en/docs/tweets/search/overview/standard). So the results you see in the website when using `Twitter’s Advanced Search` will not per se be in the result of the API search. 



## Query_01:

`All these words`: picnic, jumbo, ah, covid

`Dates:`

    From: February - 25 -2020
    To:   June - 21 -2020

**Result:** `https://twitter.com/search?q=picnic%2C%20jumbo%2C%20ah%2C%20covid%20until%3A2020-06-21%20since%3A2020-02-25&src=typed_query`

In [ ]:
query_01 = 'q=picnic%2C%20jumbo%2C%20ah%2C%20covid%20until%3A2020-06-21%20since%3A2020-02-25'
result = TweetMiner.search_tweets(raw_query = query_01)
len(result)

Expected, since the 2 Tweets there are from March and therefore will not be catch by the API.

## Query_02:

So let's make a query without `covid`.

`All these words`: picnic, jumbo, ah

**Result:** `https://twitter.com/search?q=picnic%2C%20jumbo%2C%20ah&src=typed_query`

In [ ]:
query_02 = 'q=picnic%2C%20jumbo%2C%20ah'
result_02 = TweetMiner.search_tweets(max_pages = 20, count = 100, raw_query = query_02)
len(result_02)

In [ ]:
result_02[0]

In [ ]:
df_query_02 = process_and_save(pd.DataFrame(result_02),"query_02",mine_user_twitter=0)
df_query_02.head()

In [ ]:
df_query_02.info()

## Query_03:

Adding `albertheijn` because sometimes there are expressions like `ahhhhh` and this is not what we are looking for.

`All these words`: picnic, jumbo, ah, albertheijn

**Result:** `https://twitter.com/search?q=picnic%2C%20jumbo%2C%20ah%2C%20albertheijn&src=typed_query`

In [ ]:
query_03 = 'q=picnic%2C%20jumbo%2C%20ah%2C%20albertheijn'
result_03 = TweetMiner.search_tweets(max_pages = 20, raw_query = query_03)
len(result_03)

In [ ]:
result_03[0]

In [ ]:
df_query_03 = process_and_save(pd.DataFrame(result_03),"query_03",mine_user_twitter=0)
df_query_03.head()

In [ ]:
df_query_03.info()

## Query_04:

`All these words`: picnic covid

**Result:** `https://twitter.com/search?q=picnic%20covid&src=typed_query`

In [ ]:
query_04 = 'q=picnic%20covid'
result_04 = TweetMiner.search_tweets(max_pages = 20, raw_query = query_04)
len(result_04)

In [ ]:
result_04[0]

In [ ]:
df_query_04 = process_and_save(pd.DataFrame(result_04),"query_04",mine_user_twitter=0)
df_query_04.head()

In [ ]:
df_query_04.info()

## Query_05:

`All these words`: JumboSupermarkt covid

**Result:** `https://twitter.com/search?q=JumboSupermarkt%20covid&src=typed_query`

In [ ]:
query_05 = 'q=JumboSupermarkt%20covid'
result_05 = TweetMiner.search_tweets(max_pages = 20, raw_query = query_05)
len(result_05)

In [ ]:
result_05[0]

In [ ]:
df_query_05 = process_and_save(pd.DataFrame(result_05),"query_05",mine_user_twitter=0)
df_query_05.head()

In [ ]:
df_query_05.info()

## Query_06:

`All these words`: albertheijn covid ah

**Result:** `https://twitter.com/search?q=albertheijn%20covid&src=typed_query`

In [ ]:
query_06 = 'q=albertheijn%20covid'
result_06 = TweetMiner.search_tweets(max_pages = 20, raw_query = query_06)
len(result_06)

In [ ]:
result_06[0]

In [ ]:
df_query_06 = process_and_save(pd.DataFrame(result_06),"query_06",mine_user_twitter=0)
df_query_06.head()

In [ ]:
df_query_06.info()